<a href="https://colab.research.google.com/github/jimmy-pink/colab-playground/blob/main/pre-trained/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用VGG16预训练模型

### 需求描述
有一批约800张文件夹图标的png图片，需要将这批图片分类：  

- 按形状分
  - 标准
  - 折叠1
  - 折叠2
  - 折叠3
  - 折叠4
  - 长方形-长
  - 长方形-短
  - 文件堆叠

- 按用途
  - 仅纯净背景
  - 带有前置主体对象的图标

### 面临挑战

按用途分类时的挑战
1. 许多被标记为纯净的图案实际并不纯净， 上面一般会有均匀的形状 纹理点缀
2. 有的形状较为特征， 为半折叠的文件 如图片，这种作为纯净背景时，主体一般镶嵌在中间，如何让机器理解这类图片哪种是纯净，哪种是已镶嵌好像有点困难

In [ ]:
!pip install tensorflow==2.16.2 matplotlib==3.9.1
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False